<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">

# Derivatives Analytics with Python
**&mdash;Part IIIa: Market-Based Valuation (Ch.&nbsp;8&ndash;10)**

&copy; Dr. Yves J. Hilpisch | The Python Quants

<a href="https://tpq.io" target="_blank">tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">linktr.ee/dyjh</a>

<img src="https://hilpisch.com/dawp_cover_small.png" width=30% align=left>

## Part IIIa &mdash; Market-Based Valuation

### Chapter 10 &mdash; Monte Carlo Simulation

This notebook studies discretization and simulation of the BCC building blocks, with a focus on square-root diffusions. It compares exact simulation to robust Euler schemes and illustrates convergence for European option pricing.

- Simulate the CIR short-rate model and value zero-coupon bonds by Monte Carlo.
- Compare exact sampling to the full truncation Euler discretization.
- Simulate the Heston stochastic volatility model and study Monte Carlo convergence.

Reference scripts for the book are available under `x_store/dawp/python36`.


### 1. Environment check

We start by importing the core libraries for this class and by printing version
information to confirm that a modern Python environment is active.

In [ ]:
import sys  # access basic runtime information
from pathlib import Path  # path handling for data and figure export

import math  # elementary math functions

import numpy as np  # numerical arrays
import pandas as pd  # tabular data handling
import matplotlib as mpl  # matplotlib configuration
import matplotlib.pyplot as plt  # plotting

from numpy.fft import fft  # fast Fourier transform
from numpy.random import default_rng  # random number generator

from scipy.integrate import quad  # numerical integration
import scipy.optimize as sop  # numerical optimization

np.set_printoptions(precision=6, suppress=True)  # compact numeric output

print(sys.version.split()[0])  # Python version string
print("NumPy:", np.__version__)  # NumPy version string
print("pandas:", pd.__version__)  # pandas version string

FIG_SAVE = True  # set to True to export figures as PDFs
FIG_DIR = Path("../figures")  # figure output directory
FIG_DPI = 300  # target resolution for exported figures
FIG_DISPLAY = "svg"  # inline display format: "svg" or "png"

plt.style.use("seaborn-v0_8")  # readable plotting defaults
mpl.rcParams["figure.figsize"] = (8.0, 4.5)  # consistent figure size
mpl.rcParams["axes.grid"] = True  # show a grid for readability
mpl.rcParams["savefig.dpi"] = FIG_DPI  # default export resolution

try:
    from matplotlib_inline.backend_inline import (  # Jupyter helper
        set_matplotlib_formats,
    )
    set_matplotlib_formats(FIG_DISPLAY)  # configure inline plot rendering
except Exception:
    pass

if FIG_DISPLAY == "png":
    mpl.rcParams["figure.dpi"] = FIG_DPI  # high-resolution inline rendering


def maybe_save(fig, filename):
    # Optionally saves a Matplotlib figure as a PDF file.
    if not FIG_SAVE:
        return
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    path = FIG_DIR / f"{filename}.pdf"
    fig.savefig(path, format="pdf", dpi=FIG_DPI)
    print(f"saved: {path}")
# Allow importing companion modules from ../code.
CODE_DIR = Path('..') / 'code'  # companion module directory
sys.path.insert(0, str(CODE_DIR.resolve()))  # prefer local modules



### 2. CIR short-rate simulation

The CIR model has an exact transition law in terms of a non-central chi-square distribution. For large-scale simulations, a robust alternative is the full truncation Euler scheme, which preserves non-negativity by truncating negative intermediate states.


### 3. Simulated short-rate paths

The next code cell generates a sample of CIR paths under the exact scheme and exports Figure `dawp_pIIIa_fig05_cir_paths`.


In [ ]:
import numpy as np  # numerical arrays
import matplotlib.pyplot as plt  # plotting

from dawp_pIIIa_ch10_cir_mcs import cir_generate_paths_exact  # CIR simulation

r0, kappa_r, theta_r, sigma_r = 0.01, 0.10, 0.03, 0.20
T, n_steps, n_paths = 2.0, 250, 20

r = cir_generate_paths_exact(
    r0,
    kappa_r,
    theta_r,
    sigma_r,
    T,
    n_steps=n_steps,
    n_paths=n_paths,
    seed=10,
)

t = np.linspace(0.0, T, n_steps + 1)

fig, ax = plt.subplots()
ax.plot(t, r, lw=1.2)
ax.set_xlabel('time $t$ (years)')
ax.set_ylabel('short rate $r_t$')
ax.set_title('CIR (1985): twenty simulated short-rate paths (exact scheme)')

maybe_save(fig, 'dawp_pIIIa_fig05_cir_paths')
plt.show()


### 4. ZCB valuation by Monte Carlo

A Monte Carlo ZCB estimator discounts the terminal value step-by-step along each rate path. The next cell compares analytical CIR values to Monte Carlo estimates from the exact scheme and from the Euler full truncation scheme and exports Figure `dawp_pIIIa_fig06_zcb_mcs_vs_analytical`.


In [ ]:
import numpy as np  # numerical arrays
import matplotlib.pyplot as plt  # plotting

from dawp_pIIIa_ch09_cir_zcb import cir_zcb_value  # analytical values
from dawp_pIIIa_ch10_cir_mcs import (
    cir_generate_paths_exact,
    cir_generate_paths_full_truncation,
    zcb_value_mcs,
)

r0, kappa_r, theta_r, sigma_r = 0.01, 0.10, 0.03, 0.20
T, n_steps, n_paths = 2.0, 50, 50_000

r_exact = cir_generate_paths_exact(
    r0,
    kappa_r,
    theta_r,
    sigma_r,
    T,
    n_steps=n_steps,
    n_paths=n_paths,
    seed=10,
)

r_euler = cir_generate_paths_full_truncation(
    r0,
    kappa_r,
    theta_r,
    sigma_r,
    T,
    n_steps=n_steps,
    n_paths=n_paths,
    seed=10,
)

mcs_exact = zcb_value_mcs(r_exact, T=T)
mcs_euler = zcb_value_mcs(r_euler, T=T)

t_grid = np.linspace(0.0, T, n_steps + 1)
ana = np.empty_like(t_grid)
for i, t in enumerate(t_grid):
    if t >= T:
        ana[i] = 1.0
    else:
        ana[i] = cir_zcb_value(r0, kappa_r, theta_r, sigma_r, T=T, t=float(t))

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(t_grid, ana, lw=2.0, label='analytical')
ax[0].plot(t_grid, mcs_exact, lw=0.0, marker='o', ms=3.5, label='MCS (exact)')
ax[0].plot(t_grid, mcs_euler, lw=0.0, marker='x', ms=3.8, label='MCS (Euler)')
ax[0].set_ylabel('ZCB value $B_t(T)$')
ax[0].set_title('CIR (1985) ZCB valuation for maturity $T=2$')
ax[0].legend(loc=0)

ax[1].bar(t_grid - 0.01, mcs_exact - ana, width=0.02, label='exact - analytical')
ax[1].bar(t_grid + 0.01, mcs_euler - ana, width=0.02, label='Euler - analytical')
ax[1].set_xlabel('valuation time $t$')
ax[1].set_ylabel('difference')
ax[1].legend(loc=0)

fig.tight_layout()
maybe_save(fig, 'dawp_pIIIa_fig06_zcb_mcs_vs_analytical')
plt.show()


### 5. Heston model simulation and European calls

The Heston model couples a lognormal index process with a square-root variance process. The next cell compares the Monte Carlo estimator for a European call to the Lewis (2001) benchmark price and exports Figure `dawp_pIIIa_fig07_h93_mcs_convergence`.


In [ ]:
import numpy as np  # numerical arrays
import matplotlib.pyplot as plt  # plotting

from dawp_pIIIa_ch09_bcc_option_valuation import h93_call_value_int  # benchmark
from dawp_pIIIa_ch10_h93_mcs import h93_call_value_mcs  # MCS estimator

S0, K, T, r = 100.0, 100.0, 1.0, 0.05
kappa_v, theta_v, sigma_v, rho, v0 = 1.5, 0.02, 0.30, -0.7, 0.04
n_steps = 250

ref = h93_call_value_int(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0)

paths_grid = np.array([2_500, 5_000, 10_000, 25_000, 50_000, 100_000])
vals = np.array([
    h93_call_value_mcs(
        S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0,
        n_steps=n_steps,
        n_paths=int(n_paths),
        seed=10,
    )
    for n_paths in paths_grid
])

rel_err = (vals - ref) / ref

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(paths_grid, 0.0 * paths_grid + ref, lw=2.0,
          label='Lewis (benchmark)')
ax[0].plot(paths_grid, vals, lw=0.0, marker='o', ms=5,
          label='Monte Carlo')
ax[0].set_ylabel('call value')
ax[0].set_title('Heston (1993) call values: benchmark vs Monte Carlo')
ax[0].legend(loc=0)

ax[1].plot(paths_grid, rel_err, lw=2.0, color='tab:green',
          label='relative error')
ax[1].axhline(0.0, lw=1.2, color='k')
ax[1].set_xscale('log')
ax[1].set_xlabel('number of paths')
ax[1].set_ylabel('relative error')
ax[1].legend(loc=0)

fig.tight_layout()
maybe_save(fig, 'dawp_pIIIa_fig07_h93_mcs_convergence')
plt.show()


### 6. Summary

Monte Carlo simulation turns a risk-neutral model into a flexible valuation engine, but it requires careful discretization and diagnostics. Exact simulation is a valuable benchmark where available, while robust Euler schemes allow efficient simulation in higher-dimensional settings such as stochastic volatility models.


<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">
